In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from joblib import dump
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv(r"E:\Kaggle datasets\ep_12_preprocessed.csv")
df.head(5)

,Unnamed: 0,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.00000,5.0,1.703345e+09,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Employed,15.569731,Rural,...,1.0,12.0,694.00000,2.0,1.686583e+09,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,592.92435,3.0,1.696087e+09,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Employed,10.938144,Rural,...,1.0,0.0,367.00000,1.0,1.718206e+09,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.00000,4.0,1.638372e+09,Poor,Yes,Weekly,House,2022.0


In [3]:
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
one_hot_cols = [1, 3, 5, 6, 8, 9, 15, 16, 17, 18]
sc_cols_input = [0, 2, 4, 7, 10, 11, 12, 13, 14]

sc_X = StandardScaler()
sc_y = StandardScaler()
one_hot = OneHotEncoder()

ct = ColumnTransformer(
    transformers=[
        ("standard_scaler", sc_X, sc_cols_input),
        ("onehot", one_hot, one_hot_cols)
    ],
    remainder="passthrough",
    n_jobs=-1
)

In [7]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [8]:
y_train = sc_y.fit_transform(y_train.reshape(-1, 1))

In [9]:
X_train.shape

(960000, 39)

In [10]:
y_train[0]

array([1.48357905])

In [11]:
# lr = Lasso()
#alpha = 0.3, 0.6, 0.9, 1
#max_iter = 500, 1000, 1500
#selection = random, cyclic

In [12]:
# params = [
#     {
#         "alpha": [0.3, 0.6, 0.9, 1],
#         "max_iter": [500, 1000, 1500],
#         "selection": ["random", "cyclic"]
#     }
# ]

# gs = GridSearchCV(
#     estimator=lr,
#     param_grid=params,
#     scoring='neg_mean_squared_error',
#     cv = 10
# )

In [14]:
# gs.fit(X_train, y_train)

In [31]:
# gs.best_params_

{'alpha': 0.3, 'max_iter': 500, 'selection': 'random'}

In [32]:
# gs.best_score_

-1.0000041346228958

In [15]:
best_lr = Lasso(
    alpha=0.3,
    max_iter=500,
    selection="random"
)

In [16]:
best_lr.fit(X_train, y_train)

Lasso(alpha=0.3, max_iter=500, selection='random')

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = sc_y.inverse_transform(best_lr.predict(X_test).reshape(-1, 1))

In [19]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mse {mse}")
print(f"Mae {mae}")
print(f"r2 {r2}")

Mse 747285.6247218807
Mae 668.6049133490279
r2 -5.16502856040546e-08
